In [34]:
import os
import requests
import zipfile
import geopandas as gpd

In [35]:
data_dir = "data/external/"

In [11]:
# GRWL summary stats
# https://zenodo.org/records/1297434
url = 'https://zenodo.org/records/1297434/files/GRWL_summaryStats_V01.01.zip?download=1'
out_fn = data_dir + 'GRWL_summaryStats_V01.01.zip'
response = requests.get(url)
if response.status_code == 200:
    open(out_fn, 'wb').write(response.content)
    
# unzip
with zipfile.ZipFile(out_fn, "r") as zip_ref:
    zip_ref.extractall(data_dir + 'GRWL_summaryStats_V01')

# remove zipfile
os.remove(out_fn)

In [18]:
# HydroRIVERS
# https://www.hydrosheds.org/products/hydrorivers
url = 'https://data.hydrosheds.org/file/HydroRIVERS/HydroRIVERS_v10_as_shp.zip'
out_fn = data_dir + 'HydroRIVERS_v10_as_shp.zip'
response = requests.get(url)
if response.status_code == 200:
    open(out_fn, 'wb').write(response.content)
    
# unzip
with zipfile.ZipFile(out_fn, "r") as zip_ref:
    zip_ref.extractall(data_dir) # it will unzip into a directory

# remove zipfile
os.remove(out_fn)

In [17]:
# asia polygons (for visualization)
# in this case, just load directly into memory and only 
# save a simplified version
url = 'https://stacks.stanford.edu/file/druid:yg089df0008/data.zip'
out_fn = data_dir + 'asia_polygons.gpkg'

asia_polygons = gpd.read_file(url)
asia = asia_polygons.to_crs('EPSG:32648')
asia['geometry'] = asia.simplify(1000)
asia = asia.to_crs(asia_polygons.crs)
asia = asia[asia['REGION'] == 'ASIA']
asia.to_file(out_fn)

In [4]:
# JRC yearly water classification
url = 'https://storage.googleapis.com/global-surface-water/downloads2021/occurrence/occurrence_100E_20Nv1_4_2021.tif'
out_fn = data_dir + 'occurrence_100E_20Nv1_4_2021.tif'
response = requests.get(url)
if response.status_code == 200:
            #Write to disk
            open(out_fn, 'wb').write(response.content)




In [33]:
# Tonle Sap Lake
# from https://data.opendevelopmentmekong.net/dataset/water-bodies-in-cambodia
url = 'https://data.opendevelopmentcambodia.net/en/dataset/3c74812b-0a78-4407-a072-6f91c58de45b/resource/a76793df-76aa-469a-97bc-ae3a04b6faad/download/water-bodies.geojson'
out_fn = 'data/' + 'tonle_sap_lake.geojson'
lakes = gpd.read_file(url)
tsl = lakes.loc[lakes['name'] == 'Boeung Tonle Sap']
tsl.to_file(out_fn)

In [61]:
# LMB (just remove China from Mekong shapefile)
countries = gpd.read_file(data_dir + 'asia_polygons.gpkg')
lmb_countries = [
    'VIETNAM',
    'THAILAND',
    'CAMBODIA', 
    'LAOS',
    'BURMA'
]
mekong = gpd.read_file('data/mekong.geojson')
lmb = mekong.clip(countries[countries['NAME'].isin(lmb_countries)])
lmb.to_file('data/lmb.geojson')